In [1]:
import pandas as pd
import os
import csv

In [2]:
def process_logsRe():
    root = 'C:\\bigData\\datacon\\attack_analysis\\logsRe'
    file_names = os.listdir(root)
    file_ob_list = []
    for file_name in file_names:
        file_ob = root + '\\' + file_name
        file_ob_list.append(file_ob)
    #处理所有经正则匹配后的日志后存放的dataframe
    df = pd.DataFrame({'ip': '000.000.000.000','msg':'MSG','score':0}, index=['0'])
    for file_ob in file_ob_list:
        print(file_ob)
        logs = pd.read_csv(file_ob,delimiter=',',encoding='utf-8')
        logs_ip = logs.groupby('ip').sum()
        for index,log_ip in logs_ip.iterrows():
            ip=index
            msgdatas = []
            if log_ip['threat_level']>0:
                temp_msg=logs[logs.ip==ip]['msg'].value_counts()
                test_msg=pd.DataFrame(temp_msg)
                log_ip_msg = set()
                for index,msg in test_msg.iterrows():
                    msg_list =index.split('msg');
                    for k in range(0, len(msg_list)):
                        if(msg_list[k].find(':')>-1):
                            log_ip_msg.add(msg_list[k].strip())
                for i in log_ip_msg:
                    mydict = {}
                    mydict["ip"] = ip
                    mydict["msg"] = i
                    mydict["score"] = 0
                    for index,msgs in test_msg.iterrows():
                        if(index.find(i)>-1):
                            mydict["score"] = mydict["score"]+msgs['msg']
                    msgdatas.append(mydict)
            else:
                mydict = {}
                mydict["ip"] = ip
                mydict["msg"] = 'MSG'
                mydict["score"] = 0
                msgdatas.append(mydict)
            msgData=pd.DataFrame(msgdatas)
            #print(msgData)
            df = pd.concat([df,msgData],axis=0,ignore_index=True)
    df.drop([0, 1],inplace=True)
    df.to_csv('last_result_log.csv',index=0)

In [3]:
process_logsRe()

C:\bigData\datacon\attack_analysis\logsRe\2018-12-01re.csv


C:\Users\Simpl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


C:\bigData\datacon\attack_analysis\logsRe\2018-12-02re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-03re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-04re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-05re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-06re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-07re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-08re.csv


C:\Users\Simpl\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


C:\bigData\datacon\attack_analysis\logsRe\2018-12-09re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-10re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-11re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-12re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-13re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-14re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-15re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-16re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-17re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-18re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-19re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-20re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-21re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-22re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-23re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-24re.csv
C:\bigData\datacon\attack_analysis\logsRe\2018-12-25re.c

In [4]:
def process_log_ip():
    ip_logs = pd.read_csv('last_result_log.csv',delimiter=',',encoding='utf-8')
    #将记录按ip和msg为索引计算score的总和
    ip_logs_1=ip_logs.groupby(['ip','msg']).sum()
    #重置索引
    ip_logs_1=ip_logs_1.reset_index()
    #写入logs_ip.csv为分析用户行为提供数据
    ip_logs_1.to_csv('logs_ip.csv',index=0)
    #根据ip合并字符串msg和整型计数score的合计，
    test_log=ip_logs_1.groupby(by='ip').apply(lambda x:x.sum())
    #删除多余的ip合计
    test_log.drop(['ip'],axis=1,inplace=True)
    test_log=test_log.reset_index()
    #新增和更改列名，修改非攻击者的msg，使之符合提交文件格式
    test_log['is_attacker']=test_log['score'].apply(lambda x: 1 if x>0 else 0)
    test_log['msg']=test_log['msg'].apply(lambda x: x if x!='MSG' else '')
    test_log.rename(columns={'ip':'file_id','msg':'reason'},inplace=True)
    ip_logs_1.to_csv('logs_ip_score.csv',index=0)
    test_log.drop(['score'],axis=1,inplace=True)
    print(test_log)
    order = ['file_id','is_attacker','reason']
    test_log = test_log[order]
    test_log.to_csv('result.csv',index=0)

In [5]:
process_log_ip()

               file_id                                             reason  \
0          1.10.187.55                                                      
1          1.10.188.54                                                      
2         1.180.18.194                                                      
3          1.20.100.18                                                      
4          1.223.26.13                                                      
5          1.29.160.83                                                      
6           1.68.63.45  :'PHP Injection Attack: PHP Open Tag Found',:'...   
7           1.68.63.46                                                      
8            1.80.2.56                                                      
9          1.80.234.85  :'PHP Injection Attack: High-Risk PHP Function...   
10        1.80.235.171  :'PHP Injection Attack: High-Risk PHP Function...   
11          1.85.2.113  :'PHP Injection Attack: Low-Value PHP Function...   

In [ ]:
def get_mpst_logsRe():
    root = 'C:\\bigData\\datacon\\attack_analysis\\logsRe'
    file_names = os.listdir(root)
    file_ob_list = []
    for file_name in file_names:
        file_ob = root + '\\' + file_name
        file_ob_list.append(file_ob)
    #处理所有经正则匹配后的日志后存放的dataframe
    df = pd.DataFrame(columns=('ip','date','host','method','post_data','parameter','url_path','user_agent','cookie','x-forwarded-for',
            'host_country','host_province','host_city','ip_country','ip_province','ip_city','threat_level','msg'))
    for file_ob in file_ob_list:
        logs = pd.read_csv(file_ob,delimiter=',',encoding='utf-8')
        most_logs = logs[logs['ip'].isin(most_mid_friends)]
        df = pd.concat([df,most_logs],axis=0,ignore_index=True)
    return df
most_logs=get_mpst_logsRe()
most_logs.to_csv('ip_file\\most_attacker_log.csv',index=True)